In [1]:
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
!pip install pandas


In [5]:
import pandas as pd


In [6]:
quantitative_counts_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/quantitative_counts.csv'
indicator_counts_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/indicator_counts_with_sum.csv'

In [7]:
esg_report_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/esg_report.csv'
database_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/database.csv'

In [8]:
lookup_file_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/company_id_alias.csv'

In [9]:
output_file_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/database.csv'


In [ ]:
indicator_counts_df = pd.read_csv(indicator_counts_path)
quantitative_counts_df = pd.read_csv(quantitative_counts_path)
esg_report_df = pd.read_csv(esg_report_path)

def rename_columns(df):
    df.columns = [col.lower() for col in df.columns]  # Convert all column names to lowercase
    if 'published year' in df.columns:
        df = df.rename(columns={'published year': 'year'})
    return df

indicator_counts_df = rename_columns(indicator_counts_df)
quantitative_counts_df = rename_columns(quantitative_counts_df)
esg_report_df = rename_columns(esg_report_df)

common_columns = list(set(indicator_counts_df.columns) & set(quantitative_counts_df.columns) & set(esg_report_df.columns))

merged_df = indicator_counts_df
if common_columns:
    merged_df = merged_df.merge(quantitative_counts_df, on=common_columns, how='outer', suffixes=('', '_duplicate'))
    merged_df = merged_df.merge(esg_report_df, on=common_columns, how='outer', suffixes=('', '_duplicate'))

merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_duplicate')]

merged_df.to_csv(database_path, index=False)

print(f"File saved successfully to {database_path}")

In [ ]:
main_df = pd.read_csv(database_path)
lookup_df = pd.read_csv(lookup_file_path, encoding='latin1')

lookup_df = lookup_df[['company_id', 'industry', 'company_name']]

main_df['qualitative_score'] = main_df['qualitative_sentences'] / main_df['total sentences']
main_df['quantitative_score'] = main_df['quantitative_sentences'] / main_df['total sentences']

merged_df = main_df.merge(lookup_df, on='company_id', how='left')

merged_df.to_csv(output_file_path, index=False)

print(f"Updated file saved to {output_file_path}")